# 📘 Notebook 1 — Cloud Native (Theory) + Reference Architecture (conceptual focus)

> Goal: establish a common language, link the **Twelve-Factor App** principles to what already exists in **TaskFlow**, and document at a **conceptual** level the **X-API-KEY** security, **observability** (Actuator), and **profiles/env configuration**.  
> **No execution or tests here** — hands‑on work happens in **Notebook 2 (Phase 0)** and **Notebook 3 (Gateway / Phase 1)**.

---

## 🎯 Learning outcomes

By the end, you should be able to:
- Explain Cloud Native through the **12 Factors** and point to **where** each one appears in TaskFlow.
- Describe the **domain** and **API contracts (MVP)** of TaskFlow.
- Understand **where** the **`X-API-KEY`** is configured (in the **Gateway**) and **how** it behaves.
- Understand **what** we expose for **observability** (Actuator) and **why**.
- Know the intention of **profiles** and **environment variables** that we will use in the following notebooks.

---

## 1) 🧱 The Twelve-Factor App — from concept to TaskFlow reality

> For each factor: **What it is → Phase 0 (now) → Phase 1 (now) → Next phases (Modules II–IV)**

### 1. Codebase — *one codebase, multiple deploys*
- **What it is:** one codebase per service; the same code is deployed to multiple environments (dev/test/prod).
- **Phase 0:** codebase for `tasks-service` (Spring Boot) versioned and organized by **phases/tags**.
- **Phase 1:** the `gateway` enters as an additional service; the **phase/tag** approach continues (e.g., `phase-0`, `phase-1`).
- **Next:** new services (e.g., `projects-service`) follow the same pattern.

### 2. Dependencies — *explicit and isolated*
- **What it is:** declare dependencies in Maven/Gradle; avoid relying on anything “implicit” on the host OS.
- **Phase 0:** Spring Boot + Web + Data JPA + H2 declared in `pom.xml` (no hidden/global deps).
- **Phase 1:** the gateway adds Spring Cloud Gateway dependencies.
- **Next:** add drivers/clients (e.g., PostgreSQL, observability) per profile/environment as needed.

### 3. Configuration — *in the environment, not in code*
- **What it is:** parameters/secrets via **environment variables** or per‑environment properties; do not hard‑code.
- **Phase 0:** profiles drive local behavior (e.g., port, H2).
- **Phase 1:** the **Gateway** reads an **API key** from an **environment variable** (name defined in the gateway config). When present, requests must include the `X-API-KEY` header.
- **Next:** centralized configuration (e.g., Config Server) and secret management (e.g., Secret Manager/Key Vault).

### 4. Backing services — *treat resources as attached services*
- **What it is:** DB/queues/caches treated as replaceable resources; swap without code changes.
- **Phase 0:** **H2 in‑memory** for rapid development.
- **Phase 1:** still H2 for the backend; the gateway just forwards requests.
- **Next:** move to **PostgreSQL** (on‑prem/cloud) by changing env/profiles (e.g., connection URL), not code.

### 5. Build, Release, Run — *strict separation of stages*
- **What it is:** build the artifact, combine with config, then run; stages are distinct and automatable.
- **Phase 0:** build and run the `tasks-service`; Docker option separates build/run.
- **Phase 1:** orchestrate **gateway + tasks-service** with **Docker Compose**; config remains externalized via env.
- **Next:** CI/CD pipeline to formalize build → test → image → deploy.

### 6. Processes — *stateless; store state in backing services*
- **What it is:** the service does not rely on local disk/session; persistent state belongs to the DB.
- **Phase 0:** state in H2; the service does not persist on container disk.
- **Phase 1:** same principle; the gateway is stateless (validation/routing only).
- **Next:** real persistence with PostgreSQL; external storage for files if/when introduced.

### 7. Port binding — *the app exports HTTP by itself*
- **What it is:** each service binds to a port and exposes HTTP without an external web server.
- **Phase 0:** `tasks-service` exposes HTTP (port **8081**).
- **Phase 1:** the **gateway** exposes HTTP (port **8080**) and forwards to **8081**.
- **Next:** ingress/load balancer once orchestration is used.

### 8. Concurrency — *scale out via the process model*
- **What it is:** scale by running more instances rather than stretching a single process.
- **Phase 0:** single instance of `tasks-service` in dev.
- **Phase 1:** one instance of gateway + one of backend (Compose locally).
- **Next:** multiple replicas per service behind the gateway and load‑balancing (when orchestration applies).

### 9. Disposability — *fast startup/shutdown; graceful behavior*
- **What it is:** quick start/stop; tolerate restarts and failures gracefully.
- **Phase 0:** Spring Boot starts fast; graceful shutdown by default.
- **Phase 1:** same for the gateway.
- **Next:** readiness/liveness for orchestrators; restart policies.

### 10. Dev/Prod parity — *keep environments as similar as possible*
- **What it is:** minimize differences across environments to avoid “works on my machine” issues.
- **Phase 0:** profiles (e.g., `dev`) align local behavior.
- **Phase 1:** **Docker Compose** aligns a local multi‑service setup.
- **Next:** IaC, consistent migrations and parameterization for production.

### 11. Logs — *treat logs as event streams*
- **What it is:** write to stdout/stderr; external systems aggregate/ship logs.
- **Phase 0:** basic Spring Boot logs to stdout.
- **Phase 1:** separate logs for gateway and backend.
- **Next:** correlation (request‑id), centralization, and structured logs.

### 12. Admin processes — *run administrative tasks as one‑off processes*
- **What it is:** scripts/migrations/seeds executed separately from the long‑running app.
- **Phase 0:** simple dev seed/automation may exist to ease local trials.
- **Phase 1:** same approach; the gateway needs no admin tasks.
- **Next:** **Flyway** (or similar) migrations and maintenance routines when persistent DB is introduced.

---

## 2) 🧩 Domain & requirements (MVP)

**Entities**
- **Task**: `id`, `title*`, `description`, `status {TODO|DOING|DONE}`, `projectId`, `assignee`, `createdAt`, `updatedAt`
- **Project** (optional in MVP): `id`, `name`

**Rules**
- `title` is **required**; `status` starts at `TODO`.
- Valid transitions: `TODO → DOING → DONE` (regression to `DOING` can be allowed later for demos).

**User Stories**
- **US‑1**: create task (`title`, `description`, `projectId`, `assignee`)
- **US‑2**: list tasks (filters: `status`, `projectId`)
- **US‑3**: update `status`
- **US‑4 (optional)**: create/list projects

---

## 3) 🔗 API contracts (MVP) — conceptual view (no execution here)

> **Phase 0:** direct access to `tasks-service` (port 8081).  
> **Phase 1:** access **via gateway** at `/api/tasks/**` (forwarding to the backend).

- **POST** `/tasks` (F0) / `/api/tasks` (F1) → `201 Created` + **Task** body  
  - Best practice: also return `Location` header of the new resource
- **GET** `/tasks` (F0) / `/api/tasks` (F1) with filters `status`, `projectId` → `200 OK`
- **GET** `/tasks/{id}` (F0) / `/api/tasks/{id}` (F1) → `200 OK` or `404 Not Found`
- **PATCH** `/tasks/{id}/status` (F0) / `/api/tasks/{id}/status` (F1) → `200 OK` (or `409/422` for invalid transition, per policy)

**Error payload (example)**
```json
{
  "timestamp": "2025-10-03T10:00:00Z",
  "path": "/api/tasks/...",
  "status": 400,
  "error": "Bad Request",
  "code": "VALIDATION_ERROR",
  "message": "title must not be empty"
}
```

---

## 4) 🛡️ Security (MVP) with **`X-API-KEY`** — concept & behavior

> Minimal security is applied in the **Gateway** (Phase 1).  
> **No testing here** — only what it is and how it is expected to behave.

- **What it is:** a required **header** `X-API-KEY: <token>` when the gateway’s API key is **enabled** from an **environment variable** (the variable name is defined in the gateway config).
- **Where it lives (concept):** a property in the gateway configuration that reads its value from an **environment variable**; activation may be optional in dev.
- **How it behaves (expected):**  
  - **Missing header** when required → **401 Unauthorized** (credential missing),  
  - **Invalid header** → **401/403** (depending on the gateway policy),  
  - **Valid header** → the request is forwarded to `tasks-service`.

> The concrete config/validation will be exercised in **Notebook 3 (Gateway)**.

---

## 5) 🩺 Observability — what matters now

- **Actuator** (in `tasks-service` and in the gateway):
  - `GET /actuator/health` → service health (**UP/DOWN**),
  - `GET /actuator/info` → metadata (e.g., name/version, when exposed),
  - `GET /actuator/metrics` → metric catalog (used later).
- **Why this early:** quick visibility of health, helps local orchestration, and lays the foundation for metrics and tracing in later modules.

---

## 6) ⚙️ Profiles & environment configuration — how we’ll use them

- **Typical profiles:**
  - `dev` → local execution (with **H2**) and dev‑friendly logging,
  - `docker` → Docker Compose execution; properties coming from **environment variables**.
- **Environment variables (examples of intent):**
  - Service port, DB connection parameters (when PostgreSQL is introduced),
  - Gateway API key (when the `X-API-KEY` gate is desired).

> The concrete usage (commands/files) will be demonstrated in **Notebook 2 (Phase 0)** and **Notebook 3 (Gateway)**.

---

## ✅ Checkpoints (theoretical)

- [ ] I can map each **Twelve‑Factor** to **Phase 0 and Phase 1** of TaskFlow, and state what is deferred to **Modules II–IV**.  
- [ ] I can explain the **MVP** (domain + API contracts) without opening the editor.  
- [ ] I know **where** the **`X-API-KEY`** is configured (gateway via **env**) and **how** it behaves.  
- [ ] I understand why we expose **`/actuator/health`** and **`/actuator/info`** from day one.  
- [ ] I know that **profiles/env vars** will control ports, DB, and security in the next steps.

---

## 🔜 Next notebook

**Notebook 2 — Phase 0 in practice (tasks‑service)**  
- Start the service,  
- Validate **health** and MVP endpoints,  
- Prepare for the **Gateway** (Notebook 3).
